In [ ]:
# Import Dependencies
import pandas as pd
import numpy as np
import os
import glob
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,OneHotEncoder
import pandas as pd
import tensorflow as tf

In [ ]:
# map drive to retrieve datasets
from google.colab import drive
drive.mount('/content/Drive')

Drive already mounted at /content/Drive; to attempt to forcibly remount, call drive.mount("/content/Drive", force_remount=True).


In [ ]:
# map drive to retrieve datasets
from google.colab import drive
drive.mount('/content/Drive')

Drive already mounted at /content/Drive; to attempt to forcibly remount, call drive.mount("/content/Drive", force_remount=True).


In [ ]:
# import dataset
law_inc_df = pd.read_csv('/content/Drive/MyDrive/Final_Project/Resources/gun_law_provisions_2020.csv')
law_inc_df.head()

,state,year,felony,invcommitment,invoutpatient,danger,drugmisdemeanor,alctreatment,alcoholism,relinquishment,...,dvrosurrender,dvrosurrendernoconditions,dvrosurrenderdating,expartesurrender,expartesurrendernoconditions,expartesurrenderdating,dvroremoval,stalking,lawtotal,deaths_rate_per_100k
0,Alabama,2020,0,1,0,1,0,0,1,0,...,0,0,0,0,0,0,0,0,10,23.6
1,Alaska,2020,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,3,23.5
2,Arizona,2020,1,1,1,1,0,0,0,0,...,0,0,0,0,0,0,0,1,8,16.7
3,Arkansas,2020,1,1,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,11,22.6
4,California,2020,1,1,0,1,0,0,1,1,...,1,1,1,1,1,1,1,1,111,8.5


**Preprocessing the Data for a Neural Network**

In [ ]:
# determine data values
law_inc_df.dtypes

state                      object
year                        int64
felony                      int64
invcommitment               int64
invoutpatient               int64
                           ...   
expartesurrenderdating      int64
dvroremoval                 int64
stalking                    int64
lawtotal                    int64
deaths_rate_per_100k      float64
Length: 138, dtype: object

In [ ]:
# merge state and year into one column, replace as index and drop 'year' and 'state' columns
# law_inc_df["state_year"] = law_inc_df['state'] +"_"+ law_inc_df['year'].astype(str)
# law_inc_df.set_index('state', inplace = True)
law_inc_df.drop(['year'], axis=1, inplace=True)
law_inc_df

,state,felony,invcommitment,invoutpatient,danger,drugmisdemeanor,alctreatment,alcoholism,relinquishment,violent,...,dvrosurrender,dvrosurrendernoconditions,dvrosurrenderdating,expartesurrender,expartesurrendernoconditions,expartesurrenderdating,dvroremoval,stalking,lawtotal,deaths_rate_per_100k
0,Alabama,0,1,0,1,0,0,1,0,0,...,0,0,0,0,0,0,0,0,10,23.6
1,Alaska,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,3,23.5
2,Arizona,1,1,1,1,0,0,0,0,0,...,0,0,0,0,0,0,0,1,8,16.7
3,Arkansas,1,1,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,11,22.6
4,California,1,1,0,1,0,0,1,1,1,...,1,1,1,1,1,1,1,1,111,8.5
5,Colorado,1,0,0,0,0,0,0,0,0,...,1,1,0,0,0,0,0,1,32,15.4
6,Connecticut,1,1,1,1,1,0,0,1,1,...,1,1,1,1,1,1,0,1,89,6.0
7,Delaware,1,1,1,0,1,0,0,0,0,...,0,0,0,0,0,0,0,1,41,14.4
8,Florida,1,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,30,13.7
9,Georgia,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,6,17.7


In [ ]:
# retrieve list of columns
law_inc_df.columns.tolist()

['state',
 'felony',
 'invcommitment',
 'invoutpatient',
 'danger',
 'drugmisdemeanor',
 'alctreatment',
 'alcoholism',
 'relinquishment',
 'violent',
 'violenth',
 'violentpartial',
 'dealer',
 'dealerh',
 'recordsall',
 'recordsallh',
 'recordsdealer',
 'recordsdealerh',
 'reportall',
 'reportallh',
 'reportdealer',
 'reportdealerh',
 'purge',
 'residential',
 'theft',
 'security',
 'inspection',
 'ammlicense',
 'ammrecords',
 'permit',
 'permith',
 'fingerprint',
 'training',
 'permitlaw',
 'registration',
 'registrationh',
 'defactoreg',
 'defactoregh',
 'ammpermit',
 'ammrestrict',
 'age21handgunsale',
 'age18longgunsale',
 'age21longgunsaled',
 'age21longgunsale',
 'age21handgunpossess',
 'age18longgunpossess',
 'age21longgunpossess',
 'loststolen',
 'amm18',
 'amm21h',
 'universal',
 'universalh',
 'gunshow',
 'gunshowh',
 'universalpermit',
 'universalpermith',
 'backgroundpurge',
 'ammbackground',
 'threedaylimit',
 'mentalhealth',
 'statechecks',
 'statechecksh',
 'waiting',


In [ ]:
# Generate our categorical variable lists
state_cat = law_inc_df.dtypes[law_inc_df.dtypes == "object"].index.tolist()

In [ ]:
# Create a OneHotEncoder instance
enc = OneHotEncoder(sparse=False)

# Fit and transform the OneHotEncoder using the categorical variable list
encode_df = pd.DataFrame(enc.fit_transform(law_inc_df[state_cat]))

# Add the encoded variable names to the dataframe
encode_df.columns = enc.get_feature_names_out(state_cat)
encode_df.head(10)

,state_Alabama,state_Alaska,state_Arizona,state_Arkansas,state_California,state_Colorado,state_Connecticut,state_Delaware,state_Florida,state_Georgia,...,state_South Dakota,state_Tennessee,state_Texas,state_Utah,state_Vermont,state_Virginia,state_Washington,state_West Virginia,state_Wisconsin,state_Wyoming
0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
# Merge one-hot encoded features and drop the originals
law_inc_df = law_inc_df.merge(encode_df,left_index=True, right_index=True)
law_inc_df = law_inc_df.drop(state_cat,1)
law_inc_df.head()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  This is separate from the ipykernel package so we can avoid doing imports until


,felony,invcommitment,invoutpatient,danger,drugmisdemeanor,alctreatment,alcoholism,relinquishment,violent,violenth,...,state_South Dakota,state_Tennessee,state_Texas,state_Utah,state_Vermont,state_Virginia,state_Washington,state_West Virginia,state_Wisconsin,state_Wyoming
0,0,1,0,1,0,0,1,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1,0,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1,1,1,1,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1,1,0,1,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1,1,0,1,0,0,1,1,1,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
# Split our preprocessed data into our features and target arrays
y = law_inc_df.deaths_rate_per_100k
X = law_inc_df.drop(columns="deaths_rate_per_100k")

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=1)

In [ ]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

**Compile, Train and Evaluate the Model**

In [ ]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(X_train_scaled[0])
hidden_nodes_layer1 = 80
hidden_nodes_layer2 = 30

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu")
)

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 80)                14880     
                                                                 
 dense_1 (Dense)             (None, 30)                2430      
                                                                 
 dense_2 (Dense)             (None, 1)                 31        
                                                                 
Total params: 17,341
Trainable params: 17,341
Non-trainable params: 0
_________________________________________________________________


In [ ]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [ ]:
# Import checkpoint dependencies
import os
from tensorflow.keras.callbacks import ModelCheckpoint

# Define the checkpoint path and filenames
os.makedirs("checkpoints/",exist_ok=True)
checkpoint_path = "checkpoints/weights.{epoch:02d}.hdf5"


# Create a callback that saves the model's weights every epoch
cp_callback = ModelCheckpoint(
    filepath=checkpoint_path,
    verbose=1,
    save_weights_only=True,
    save_freq='epoch',
    period=5)


# Train the model
fit_model = nn.fit(X_train_scaled,
                   y_train,
                   epochs=100,
                   callbacks=[cp_callback])

Epoch 1/100
2/2 [==============================] - 1s 10ms/step - loss: -3.3644 - accuracy: 0.0000e+00
Epoch 2/100
2/2 [==============================] - 0s 6ms/step - loss: -8.8908 - accuracy: 0.0000e+00
Epoch 3/100
2/2 [==============================] - 0s 5ms/step - loss: -13.6774 - accuracy: 0.0000e+00
Epoch 4/100
2/2 [==============================] - 0s 5ms/step - loss: -18.2923 - accuracy: 0.0000e+00
Epoch 5/100
1/2 [==============>...............] - ETA: 0s - loss: -23.2877 - accuracy: 0.0000e+00
Epoch 5: saving model to checkpoints/weights.05.hdf5
2/2 [==============================] - 0s 14ms/step - loss: -22.8172 - accuracy: 0.0000e+00
Epoch 6/100
2/2 [==============================] - 0s 8ms/step - loss: -27.5857 - accuracy: 0.0000e+00
Epoch 7/100
2/2 [==============================] - 0s 4ms/step - loss: -32.7022 - accuracy: 0.0000e+00
Epoch 8/100
2/2 [==============================] - 0s 4ms/step - loss: -38.0843 - accuracy: 0.0000e+00
Epoch 9/100
2/2 [===================

In [ ]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

1/1 - 0s - loss: -1.3123e+04 - accuracy: 0.0000e+00 - 111ms/epoch - 111ms/step
Loss: -13122.65234375, Accuracy: 0.0


In [ ]:
# Save and export results to an HDF5 file
nn.save("AlphabetSoupCharity.h5")

**Optimize the Model**

Preprocess and clean for optimization

In [ ]:
law_inc_df2 = pd.read_csv('/content/Drive/MyDrive/Final_Project/Resources/gun_law_provisions_2020.csv')
law_inc_df2.head()

,state,year,felony,invcommitment,invoutpatient,danger,drugmisdemeanor,alctreatment,alcoholism,relinquishment,...,dvrosurrender,dvrosurrendernoconditions,dvrosurrenderdating,expartesurrender,expartesurrendernoconditions,expartesurrenderdating,dvroremoval,stalking,lawtotal,deaths_rate_per_100k
0,Alabama,2020,0,1,0,1,0,0,1,0,...,0,0,0,0,0,0,0,0,10,23.6
1,Alaska,2020,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,3,23.5
2,Arizona,2020,1,1,1,1,0,0,0,0,...,0,0,0,0,0,0,0,1,8,16.7
3,Arkansas,2020,1,1,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,11,22.6
4,California,2020,1,1,0,1,0,0,1,1,...,1,1,1,1,1,1,1,1,111,8.5


In [ ]:
law_inc_df2.columns.tolist()

['state',
 'year',
 'felony',
 'invcommitment',
 'invoutpatient',
 'danger',
 'drugmisdemeanor',
 'alctreatment',
 'alcoholism',
 'relinquishment',
 'violent',
 'violenth',
 'violentpartial',
 'dealer',
 'dealerh',
 'recordsall',
 'recordsallh',
 'recordsdealer',
 'recordsdealerh',
 'reportall',
 'reportallh',
 'reportdealer',
 'reportdealerh',
 'purge',
 'residential',
 'theft',
 'security',
 'inspection',
 'ammlicense',
 'ammrecords',
 'permit',
 'permith',
 'fingerprint',
 'training',
 'permitlaw',
 'registration',
 'registrationh',
 'defactoreg',
 'defactoregh',
 'ammpermit',
 'ammrestrict',
 'age21handgunsale',
 'age18longgunsale',
 'age21longgunsaled',
 'age21longgunsale',
 'age21handgunpossess',
 'age18longgunpossess',
 'age21longgunpossess',
 'loststolen',
 'amm18',
 'amm21h',
 'universal',
 'universalh',
 'gunshow',
 'gunshowh',
 'universalpermit',
 'universalpermith',
 'backgroundpurge',
 'ammbackground',
 'threedaylimit',
 'mentalhealth',
 'statechecks',
 'statechecksh',
 'w

In [ ]:
# Drop the non-beneficial ID columns, 'year'
law_inc_df2.drop(['year'], axis=1, inplace=True)

In [ ]:
# Generate our categorical variable lists
state_cat = law_inc_df2.dtypes[law_inc_df2.dtypes == "object"].index.tolist()

In [ ]:
# Create a OneHotEncoder instance
enc = OneHotEncoder(sparse=False)

# Fit and transform the OneHotEncoder using the categorical variable list
encode_df = pd.DataFrame(enc.fit_transform(law_inc_df2[state_cat]))

# Add the encoded variable names to the dataframe
encode_df.columns = enc.get_feature_names(state_cat)
encode_df.head()

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


,state_Alabama,state_Alaska,state_Arizona,state_Arkansas,state_California,state_Colorado,state_Connecticut,state_Delaware,state_Florida,state_Georgia,...,state_South Dakota,state_Tennessee,state_Texas,state_Utah,state_Vermont,state_Virginia,state_Washington,state_West Virginia,state_Wisconsin,state_Wyoming
0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
# Merge one-hot encoded features and drop the originals
law_inc_df2 = law_inc_df2.merge(encode_df,left_index=True, right_index=True)
law_inc_df2 = law_inc_df2.drop(state_cat,1)
law_inc_df2.head()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  This is separate from the ipykernel package so we can avoid doing imports until


,felony,invcommitment,invoutpatient,danger,drugmisdemeanor,alctreatment,alcoholism,relinquishment,violent,violenth,...,state_South Dakota,state_Tennessee,state_Texas,state_Utah,state_Vermont,state_Virginia,state_Washington,state_West Virginia,state_Wisconsin,state_Wyoming
0,0,1,0,1,0,0,1,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1,0,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1,1,1,1,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1,1,0,1,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1,1,0,1,0,0,1,1,1,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
# Split our preprocessed data into our features and target arrays
y = law_inc_df2.deaths_rate_per_100k
X = law_inc_df2.drop(columns="deaths_rate_per_100k")

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5, random_state=1)

In [ ]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

Optimization Attempt 1: hiddent layers, editing train model epochs

In [ ]:
# Define the model - deep neural net
number_input_features = len(X_train_scaled[0])
hidden_nodes_layer1 = 100
hidden_nodes_layer2 = 80
hidden_nodes_layer3 = 60

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu")
)

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Third hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer3, activation="relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_3 (Dense)             (None, 100)               18600     
                                                                 
 dense_4 (Dense)             (None, 80)                8080      
                                                                 
 dense_5 (Dense)             (None, 60)                4860      
                                                                 
 dense_6 (Dense)             (None, 1)                 61        
                                                                 
Total params: 31,601
Trainable params: 31,601
Non-trainable params: 0
_________________________________________________________________


In [ ]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [ ]:
# Import checkpoint dependencies
import os
from tensorflow.keras.callbacks import ModelCheckpoint

# Define the checkpoint path and filenames
os.makedirs("checkpoints_opt1/",exist_ok=True)
checkpoint_path = "checkpoints_opt1/weights.{epoch:02d}.hdf5"

# Create a callback that saves the model's weights every epoch
cp_callback = ModelCheckpoint(
    filepath=checkpoint_path,
    verbose=1,
    save_weights_only=True,
    save_freq='epoch',
    period=5)


# Train the model
fit_model = nn.fit(X_train_scaled,
                   y_train,
                   epochs=60,
                   callbacks=[cp_callback],
                   verbose=0)


Epoch 5: saving model to checkpoints_opt1/weights.05.hdf5

Epoch 10: saving model to checkpoints_opt1/weights.10.hdf5

Epoch 15: saving model to checkpoints_opt1/weights.15.hdf5

Epoch 20: saving model to checkpoints_opt1/weights.20.hdf5

Epoch 25: saving model to checkpoints_opt1/weights.25.hdf5

Epoch 30: saving model to checkpoints_opt1/weights.30.hdf5

Epoch 35: saving model to checkpoints_opt1/weights.35.hdf5

Epoch 40: saving model to checkpoints_opt1/weights.40.hdf5

Epoch 45: saving model to checkpoints_opt1/weights.45.hdf5

Epoch 50: saving model to checkpoints_opt1/weights.50.hdf5

Epoch 55: saving model to checkpoints_opt1/weights.55.hdf5

Epoch 60: saving model to checkpoints_opt1/weights.60.hdf5


In [ ]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

1/1 - 0s - loss: -6.8326e+03 - accuracy: 0.0000e+00 - 105ms/epoch - 105ms/step
Loss: -6832.62109375, Accuracy: 0.0


In [ ]:
# Export our model to HDF5 file
nn.save("AlphabetSoupCharity_optimize_one.h5")

Optimization Attempt 2: testing different activation functions

In [ ]:
# Define the model - deep neural net
number_input_features = len(X_train_scaled[0])
hidden_nodes_layer1 = 80
hidden_nodes_layer2 = 50
hidden_nodes_layer3 = 40

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="sigmoid")
)

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="sigmoid"))

# Third hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer3, activation="sigmoid"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="relu"))

# Check the structure of the model
nn.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_7 (Dense)             (None, 80)                14880     
                                                                 
 dense_8 (Dense)             (None, 50)                4050      
                                                                 
 dense_9 (Dense)             (None, 40)                2040      
                                                                 
 dense_10 (Dense)            (None, 1)                 41        
                                                                 
Total params: 21,011
Trainable params: 21,011
Non-trainable params: 0
_________________________________________________________________


In [ ]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [ ]:
# Import checkpoint dependencies
import os
from tensorflow.keras.callbacks import ModelCheckpoint

# Define the checkpoint path and filenames
os.makedirs("checkpoints_opt2/",exist_ok=True)
checkpoint_path = "checkpoints_opt2/weights.{epoch:02d}.hdf5"

# Create a callback that saves the model's weights every epoch
cp_callback = ModelCheckpoint(
    filepath=checkpoint_path,
    verbose=1,
    save_weights_only=True,
    save_freq='epoch',
    period=5)


# Train the model
fit_model = nn.fit(X_train_scaled,
                   y_train,
                   epochs=50,
                   callbacks=[cp_callback],
                   verbose=0)


Epoch 5: saving model to checkpoints_opt2/weights.05.hdf5

Epoch 10: saving model to checkpoints_opt2/weights.10.hdf5

Epoch 15: saving model to checkpoints_opt2/weights.15.hdf5

Epoch 20: saving model to checkpoints_opt2/weights.20.hdf5

Epoch 25: saving model to checkpoints_opt2/weights.25.hdf5

Epoch 30: saving model to checkpoints_opt2/weights.30.hdf5

Epoch 35: saving model to checkpoints_opt2/weights.35.hdf5

Epoch 40: saving model to checkpoints_opt2/weights.40.hdf5

Epoch 45: saving model to checkpoints_opt2/weights.45.hdf5

Epoch 50: saving model to checkpoints_opt2/weights.50.hdf5


In [ ]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

1/1 - 0s - loss: 245.6269 - accuracy: 0.0000e+00 - 120ms/epoch - 120ms/step
Loss: 245.6268768310547, Accuracy: 0.0


In [ ]:
# Export our model to HDF5 file
nn.save("AlphabetSoupCharity_optimize_two.h5")

Optimization Attempt 3: dropping columns

In [ ]:
law_inc_df3 = pd.read_csv('/content/Drive/MyDrive/Final_Project/Resources/gun_law_provisions_2020.csv')
law_inc_df3.head()

,state,year,felony,invcommitment,invoutpatient,danger,drugmisdemeanor,alctreatment,alcoholism,relinquishment,...,dvrosurrender,dvrosurrendernoconditions,dvrosurrenderdating,expartesurrender,expartesurrendernoconditions,expartesurrenderdating,dvroremoval,stalking,lawtotal,deaths_rate_per_100k
0,Alabama,2020,0,1,0,1,0,0,1,0,...,0,0,0,0,0,0,0,0,10,23.6
1,Alaska,2020,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,3,23.5
2,Arizona,2020,1,1,1,1,0,0,0,0,...,0,0,0,0,0,0,0,1,8,16.7
3,Arkansas,2020,1,1,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,11,22.6
4,California,2020,1,1,0,1,0,0,1,1,...,1,1,1,1,1,1,1,1,111,8.5


In [ ]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
law_inc_df3.drop(['year', 'liability','cap14','cap16','cap18','onefeature','threedaylimit','age18longgunsale','purge','dvrosurrenderdating','expartesurrender','expartesurrendernoconditions'], axis=1, inplace=True)

In [ ]:
# Generate our categorical variable lists
state_cat2 = law_inc_df3.dtypes[law_inc_df3.dtypes == "object"].index.tolist()

In [ ]:
# Create a OneHotEncoder instance
enc2 = OneHotEncoder(sparse=False)

# Fit and transform the OneHotEncoder using the categorical variable list
encode_df2 = pd.DataFrame(enc2.fit_transform(law_inc_df3[state_cat2]))

# Add the encoded variable names to the dataframe
encode_df2.columns = enc2.get_feature_names(state_cat2)

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [ ]:
# Merge one-hot encoded features and drop the originals
law_inc_df3 = law_inc_df3.merge(encode_df2,left_index=True, right_index=True)
law_inc_df3 = law_inc_df3.drop(state_cat2,1)
law_inc_df3.head()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  This is separate from the ipykernel package so we can avoid doing imports until


,felony,invcommitment,invoutpatient,danger,drugmisdemeanor,alctreatment,alcoholism,relinquishment,violent,violenth,...,state_South Dakota,state_Tennessee,state_Texas,state_Utah,state_Vermont,state_Virginia,state_Washington,state_West Virginia,state_Wisconsin,state_Wyoming
0,0,1,0,1,0,0,1,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1,0,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1,1,1,1,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1,1,0,1,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1,1,0,1,0,0,1,1,1,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
# Split our preprocessed data into our features and target arrays
y = law_inc_df3.deaths_rate_per_100k
X = law_inc_df3.drop(columns="deaths_rate_per_100k")

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5, random_state=1)

In [ ]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [ ]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(X_train_scaled[0])
hidden_nodes_layer1 = 80
hidden_nodes_layer2 = 30

nn_new = tf.keras.models.Sequential()

# First hidden layer
nn_new.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu")
)

# Second hidden layer
nn_new.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Output layer
nn_new.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn_new.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_11 (Dense)            (None, 80)                14000     
                                                                 
 dense_12 (Dense)            (None, 30)                2430      
                                                                 
 dense_13 (Dense)            (None, 1)                 31        
                                                                 
Total params: 16,461
Trainable params: 16,461
Non-trainable params: 0
_________________________________________________________________


In [ ]:
# Compile the model
nn_new.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [ ]:
# Import checkpoint dependencies
import os
from tensorflow.keras.callbacks import ModelCheckpoint

# Define the checkpoint path and filenames
os.makedirs("checkpoints_opt3/",exist_ok=True)
checkpoint_path = "checkpoints_opt3/weights.{epoch:02d}.hdf5"

# Create a callback that saves the model's weights every epoch
cp_callback = ModelCheckpoint(
    filepath=checkpoint_path,
    verbose=1,
    save_weights_only=True,
    save_freq='epoch',
    period=5)


# Train the model
fit_model = nn_new.fit(X_train_scaled,
                   y_train,
                   epochs=50,
                   callbacks=[cp_callback],
                   verbose=0)


Epoch 5: saving model to checkpoints_opt3/weights.05.hdf5

Epoch 10: saving model to checkpoints_opt3/weights.10.hdf5

Epoch 15: saving model to checkpoints_opt3/weights.15.hdf5

Epoch 20: saving model to checkpoints_opt3/weights.20.hdf5

Epoch 25: saving model to checkpoints_opt3/weights.25.hdf5

Epoch 30: saving model to checkpoints_opt3/weights.30.hdf5

Epoch 35: saving model to checkpoints_opt3/weights.35.hdf5

Epoch 40: saving model to checkpoints_opt3/weights.40.hdf5

Epoch 45: saving model to checkpoints_opt3/weights.45.hdf5

Epoch 50: saving model to checkpoints_opt3/weights.50.hdf5


In [ ]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn_new.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

1/1 - 0s - loss: -3.2358e+02 - accuracy: 0.0000e+00 - 84ms/epoch - 84ms/step
Loss: -323.57781982421875, Accuracy: 0.0


In [ ]:
# Save and export results to an HDF5 file
nn_new.save("AlphabetSoupCharity_optimize_three.h5")

Different Machine learning attempts

In [ ]:
# !pip install hvplot

In [ ]:
# Initial imports
import pandas as pd
import hvplot.pandas
from pathlib import Path
import plotly.express as px
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans


Output hidden; open in https://colab.research.google.com to view.

In [ ]:
# map drive to retrieve datasets
from google.colab import drive
drive.mount('/content/Drive')

Drive already mounted at /content/Drive; to attempt to forcibly remount, call drive.mount("/content/Drive", force_remount=True).


In [ ]:
# map drive to retrieve datasets
from google.colab import drive
drive.mount('/content/Drive')

Drive already mounted at /content/Drive; to attempt to forcibly remount, call drive.mount("/content/Drive", force_remount=True).


In [ ]:
# import dataset
law_inc_df = pd.read_csv('/content/Drive/MyDrive/Final_Project/Resources/gun_law_provisions_2020.csv')
law_inc_df.head(5)

,state,year,felony,invcommitment,invoutpatient,danger,drugmisdemeanor,alctreatment,alcoholism,relinquishment,...,dvrosurrender,dvrosurrendernoconditions,dvrosurrenderdating,expartesurrender,expartesurrendernoconditions,expartesurrenderdating,dvroremoval,stalking,lawtotal,deaths_rate_per_100k
0,Alabama,2020,0,1,0,1,0,0,1,0,...,0,0,0,0,0,0,0,0,10,23.6
1,Alaska,2020,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,3,23.5
2,Arizona,2020,1,1,1,1,0,0,0,0,...,0,0,0,0,0,0,0,1,8,16.7
3,Arkansas,2020,1,1,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,11,22.6
4,California,2020,1,1,0,1,0,0,1,1,...,1,1,1,1,1,1,1,1,111,8.5


In [ ]:
# drop year, replace state as index
# law_inc_df.set_index('state', inplace = True)
law_inc_df.drop(['year'], axis=1, inplace=True)

In [ ]:
law_inc_df.dtypes

state                      object
felony                      int64
invcommitment               int64
invoutpatient               int64
danger                      int64
                           ...   
expartesurrenderdating      int64
dvroremoval                 int64
stalking                    int64
lawtotal                    int64
deaths_rate_per_100k      float64
Length: 137, dtype: object

In [ ]:
# Find null values
for column in law_inc_df.columns:
    print(f"Column {column} has {law_inc_df[column].isnull().sum()} null values")

Column state has 0 null values
Column felony has 0 null values
Column invcommitment has 0 null values
Column invoutpatient has 0 null values
Column danger has 0 null values
Column drugmisdemeanor has 0 null values
Column alctreatment has 0 null values
Column alcoholism has 0 null values
Column relinquishment has 0 null values
Column violent has 0 null values
Column violenth has 0 null values
Column violentpartial has 0 null values
Column dealer has 0 null values
Column dealerh has 0 null values
Column recordsall has 0 null values
Column recordsallh has 0 null values
Column recordsdealer has 0 null values
Column recordsdealerh has 0 null values
Column reportall has 0 null values
Column reportallh has 0 null values
Column reportdealer has 0 null values
Column reportdealerh has 0 null values
Column purge has 0 null values
Column residential has 0 null values
Column theft has 0 null values
Column security has 0 null values
Column inspection has 0 null values
Column ammlicense has 0 null va

In [ ]:
# Create a new DataFrame that holds only the state names.
law_names_df = law_inc_df[['state']]
law_names_df

,state
0,Alabama
1,Alaska
2,Arizona
3,Arkansas
4,California
5,Colorado
6,Connecticut
7,Delaware
8,Florida
9,Georgia


In [ ]:
# Drop the 'state' column since it's not going to be used on the clustering elementary.
law_inc_df.drop(columns=["state"], inplace=True)
law_inc_df

,felony,invcommitment,invoutpatient,danger,drugmisdemeanor,alctreatment,alcoholism,relinquishment,violent,violenth,...,dvrosurrender,dvrosurrendernoconditions,dvrosurrenderdating,expartesurrender,expartesurrendernoconditions,expartesurrenderdating,dvroremoval,stalking,lawtotal,deaths_rate_per_100k
0,0,1,0,1,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,10,23.6
1,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,3,23.5
2,1,1,1,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,8,16.7
3,1,1,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,11,22.6
4,1,1,0,1,0,0,1,1,1,1,...,1,1,1,1,1,1,1,1,111,8.5
5,1,0,0,0,0,0,0,0,0,0,...,1,1,0,0,0,0,0,1,32,15.4
6,1,1,1,1,1,0,0,1,1,1,...,1,1,1,1,1,1,0,1,89,6.0
7,1,1,1,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,1,41,14.4
8,1,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,30,13.7
9,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,6,17.7


In [ ]:
# Use get_dummies() to create variables for text features.
X = pd.get_dummies(law_inc_df, columns=["elementary", "stalking"])
X

,felony,invcommitment,invoutpatient,danger,drugmisdemeanor,alctreatment,alcoholism,relinquishment,violent,violenth,...,expartesurrender,expartesurrendernoconditions,expartesurrenderdating,dvroremoval,lawtotal,deaths_rate_per_100k,elementary_0,elementary_1,stalking_0,stalking_1
0,0,1,0,1,0,0,1,0,0,0,...,0,0,0,0,10,23.6,1,0,1,0
1,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,3,23.5,0,1,1,0
2,1,1,1,1,0,0,0,0,0,0,...,0,0,0,0,8,16.7,1,0,0,1
3,1,1,0,1,0,0,0,0,0,0,...,0,0,0,0,11,22.6,0,1,1,0
4,1,1,0,1,0,0,1,1,1,1,...,1,1,1,1,111,8.5,0,1,0,1
5,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,32,15.4,0,1,0,1
6,1,1,1,1,1,0,0,1,1,1,...,1,1,1,0,89,6.0,0,1,0,1
7,1,1,1,0,1,0,0,0,0,0,...,0,0,0,0,41,14.4,0,1,0,1
8,1,1,0,0,0,0,0,0,0,0,...,0,0,0,0,30,13.7,0,1,1,0
9,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,6,17.7,1,0,1,0


In [ ]:
# Standardize the data with StandardScaler().
law_scaled = StandardScaler().fit_transform(X)
print(law_scaled)

[[-1.68705478  0.81649658 -0.68599434 ... -1.45773797  0.75
  -0.75      ]
 [ 0.59274898 -1.22474487 -0.68599434 ...  0.68599434  0.75
  -0.75      ]
 [ 0.59274898  0.81649658  1.45773797 ... -1.45773797 -1.33333333
   1.33333333]
 ...
 [ 0.59274898  0.81649658  1.45773797 ... -1.45773797  0.75
  -0.75      ]
 [ 0.59274898  0.81649658  1.45773797 ...  0.68599434 -1.33333333
   1.33333333]
 [ 0.59274898 -1.22474487 -0.68599434 ...  0.68599434  0.75
  -0.75      ]]


In [ ]:
# Using PCA to reduce dimension to three principal components.
pca = PCA(n_components=3)
law_pca = pca.fit_transform(law_scaled)
law_pca

array([[-4.80920836e+00,  8.12823797e-01,  4.39299272e-01],
       [-5.68113974e+00,  1.62018341e+00,  1.27452481e+00],
       [-4.56460673e+00,  7.17684703e-01,  1.10962684e-01],
       [-4.81026591e+00,  7.89925924e-01,  3.53220527e-01],
       [ 2.17035063e+01,  6.63356712e+00,  9.64791723e+00],
       [-2.97210565e-01, -3.94149681e+00,  5.74521864e-01],
       [ 1.54878027e+01, -3.25483924e+00,  2.21619924e+00],
       [ 2.13655131e+00, -4.43707062e+00, -4.32598642e-01],
       [-9.54164304e-01,  1.75891010e+00, -9.22832615e-01],
       [-5.31154528e+00,  1.22443333e+00,  3.76388927e-01],
       [ 1.31072568e+01,  4.23169968e-01, -8.18353297e+00],
       [-5.75322361e+00,  1.47872083e+00,  1.25144874e+00],
       [ 8.80213363e+00, -1.71699179e+00, -5.72309833e+00],
       [-3.89363354e+00,  3.12656818e-01,  6.58000225e-01],
       [-1.65022560e+00, -7.15220370e-01, -7.21937309e-01],
       [-4.98979003e+00,  1.45970579e+00, -2.86580935e-02],
       [-5.45209940e+00,  1.20484997e+00

In [ ]:
# Create a DataFrame with the three principal components.
pcs_df  = pd.DataFrame(
    data=law_pca, columns=["PC 1", "PC 2", "PC 3",], index = law_names_df.index)
pcs_df 

,PC 1,PC 2,PC 3
0,-4.809208,0.812824,0.439299
1,-5.681140,1.620183,1.274525
2,-4.564607,0.717685,0.110963
3,-4.810266,0.789926,0.353221
4,21.703506,6.633567,9.647917
5,-0.297211,-3.941497,0.574522
6,15.487803,-3.254839,2.216199
7,2.136551,-4.437071,-0.432599
8,-0.954164,1.758910,-0.922833
9,-5.311545,1.224433,0.376389


Finding the Best Value for k Using the Elbow Curve

In [ ]:
# Create an elbow curve to find the best value for K.
inertia = []
k = list(range(1, 11))

# Calculate the inertia for the range of K values
for i in k:
    km = KMeans(n_clusters=i, random_state=0)
    km.fit(pcs_df)
    inertia.append(km.inertia_)

# Create the elbow curve
elbow_data = {"k": k, "inertia": inertia}
df_elbow = pd.DataFrame(elbow_data)
df_elbow.hvplot.line(x="k", y="inertia", xticks=k, title="Elbow Curve")


:Curve   [k]   (inertia)

In [ ]:
# Initialize the K-Means model.
model = KMeans(n_clusters=4, random_state=0)    

# Fit the model
model.fit(pcs_df)

# Predict clusters
predictions = model.predict(pcs_df)

predictions

array([2, 2, 2, 2, 1, 0, 3, 0, 2, 2, 3, 2, 3, 2, 2, 2, 2, 2, 2, 3, 1, 2,
       0, 2, 2, 2, 2, 0, 2, 3, 2, 3, 0, 2, 2, 2, 0, 0, 0, 2, 2, 2, 2, 2,
       2, 0, 0, 2, 2, 2], dtype=int32)

In [ ]:
# Create a new DataFrame including predicted clusters and states features.
# Concatentate the law_inc_df and pcs_df DataFrames on the same columns.
    # Create return DataFrame with predicted clusters
clustered_df = pd.concat([law_inc_df,pcs_df], axis= 1, join="inner")

#  Add a new column, "state" to the clustered_df DataFrame that holds the names of the states. 
clustered_df = pd.concat([clustered_df,law_names_df], axis= 1, join="inner")

#  Add a new column, "Class" to the clustered_df DataFrame that holds the predictions.
clustered_df["Class"] = model.labels_

# Print the shape of the clustered_df
print(clustered_df.shape)
clustered_df.head(10)

(50, 141)


,felony,invcommitment,invoutpatient,danger,drugmisdemeanor,alctreatment,alcoholism,relinquishment,violent,violenth,...,expartesurrenderdating,dvroremoval,stalking,lawtotal,deaths_rate_per_100k,PC 1,PC 2,PC 3,state,Class
0,0,1,0,1,0,0,1,0,0,0,...,0,0,0,10,23.6,-4.809208,0.812824,0.439299,Alabama,2
1,1,0,0,0,0,0,0,0,0,0,...,0,0,0,3,23.5,-5.681140,1.620183,1.274525,Alaska,2
2,1,1,1,1,0,0,0,0,0,0,...,0,0,1,8,16.7,-4.564607,0.717685,0.110963,Arizona,2
3,1,1,0,1,0,0,0,0,0,0,...,0,0,0,11,22.6,-4.810266,0.789926,0.353221,Arkansas,2
4,1,1,0,1,0,0,1,1,1,1,...,1,1,1,111,8.5,21.703506,6.633567,9.647917,California,1
5,1,0,0,0,0,0,0,0,0,0,...,0,0,1,32,15.4,-0.297211,-3.941497,0.574522,Colorado,0
6,1,1,1,1,1,0,0,1,1,1,...,1,0,1,89,6.0,15.487803,-3.254839,2.216199,Connecticut,3
7,1,1,1,0,1,0,0,0,0,0,...,0,0,1,41,14.4,2.136551,-4.437071,-0.432599,Delaware,0
8,1,1,0,0,0,0,0,0,0,0,...,0,0,0,30,13.7,-0.954164,1.758910,-0.922833,Florida,2
9,1,0,0,0,0,0,0,0,0,0,...,0,0,0,6,17.7,-5.311545,1.224433,0.376389,Georgia,2


In [ ]:
# Creating a 3D-Scatter with the PCA data and the clusters
fig = px.scatter_3d(
    clustered_df,
    x="PC 1",
    y="PC 2",
    z="PC 3",
    color="Class",
    symbol="Class",
    hover_name= 'state',
    hover_data= ['elementary'],
    width=800,
)
fig.update_layout(legend=dict(x=0, y=1))
fig.show()

In [ ]:
# Create a table with  states.
clustered_df.hvplot.table(columns=['state','elementary','stalking','lawtotal','deaths_rate_per_100k','Class'], sortable=True, selectable=True)

:Table   [state,elementary,stalking,lawtotal,deaths_rate_per_100k,Class]

In [ ]:
# Print the total number of  states.
print(f"There are: {len(clustered_df)} applicable states.")

There are: 50 applicable states.


In [ ]:
# Scaling data to create the scatter plot with  states.
clustered_scaled = MinMaxScaler().fit_transform(clustered_df[['lawtotal', 'deaths_rate_per_100k']])


In [ ]:
# Create a new DataFrame that has the scaled data with the clustered_df DataFrame index.
plot_df = pd.DataFrame(
    data=clustered_scaled, columns=[
        "lawtotal",
        "deaths_rate_per_100k"],
        index=clustered_df.index.copy()
    )

# Add the "state" column from the clustered_df DataFrame to the new DataFrame.
plot_df['state'] = clustered_df.state

# Add the "Class" column from the clustered_df DataFrame to the new DataFrame. 
plot_df['Class'] = clustered_df.Class

plot_df.head(10)

,lawtotal,deaths_rate_per_100k,state,Class
0,0.081818,0.801587,Alabama,2
1,0.018182,0.797619,Alaska,2
2,0.063636,0.527778,Arizona,2
3,0.090909,0.761905,Arkansas,2
4,1.000000,0.202381,California,1
5,0.281818,0.476190,Colorado,0
6,0.800000,0.103175,Connecticut,3
7,0.363636,0.436508,Delaware,0
8,0.263636,0.408730,Florida,2
9,0.045455,0.567460,Georgia,2


In [ ]:
# Create a hvplot.scatter plot using x="lawtotal" and y="deaths_rate_per_100k".
plot_df.hvplot.scatter(
    x="lawtotal", 
    y="deaths_rate_per_100k", 
    hover_cols=["state"],
    by="Class",
)


:NdOverlay   [Class]
   :Scatter   [lawtotal]   (deaths_rate_per_100k,state)

In [ ]:
# # Saving data
# file_path = "/df_output/plot_df.csv"
# plot_df.to_csv(file_path, index=False)

# file_path = "/df_output/clustered_df.csv"
# clustered_df.to_csv(file_path, index=False)

# file_path = "/df_output/pcs_df.csv"
# pcs_df.to_csv(file_path, index=False)

# file_path = "/df_output/law_inc_df.csv"
# law_inc_df.to_csv(file_path, index=False)

# file_path = "/df_output/law_names_df.csv"
# law_names_df.to_csv(file_path, index=False)

k-means

In [ ]:
# Initial imports
import pandas as pd
from sklearn.cluster import KMeans
import plotly.express as px
import hvplot.pandas

In [ ]:
# import dataset
law_inc_df = pd.read_csv('/content/Drive/MyDrive/Final_Project/Resources/gun_law_provisions_2020.csv')
law_inc_df.head(5)

,state,year,felony,invcommitment,invoutpatient,danger,drugmisdemeanor,alctreatment,alcoholism,relinquishment,...,dvrosurrender,dvrosurrendernoconditions,dvrosurrenderdating,expartesurrender,expartesurrendernoconditions,expartesurrenderdating,dvroremoval,stalking,lawtotal,deaths_rate_per_100k
0,Alabama,2020,0,1,0,1,0,0,1,0,...,0,0,0,0,0,0,0,0,10,23.6
1,Alaska,2020,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,3,23.5
2,Arizona,2020,1,1,1,1,0,0,0,0,...,0,0,0,0,0,0,0,1,8,16.7
3,Arkansas,2020,1,1,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,11,22.6
4,California,2020,1,1,0,1,0,0,1,1,...,1,1,1,1,1,1,1,1,111,8.5


In [ ]:
law_inc_df.drop(columns=['state', 'year'], inplace=True)

In [ ]:
law_inc_df.hvplot.scatter(x="lawtotal", y="deaths_rate_per_100k")

:Scatter   [lawtotal]   (deaths_rate_per_100k)

In [ ]:
# Function to cluster and plot dataset
def test_cluster_amount(df, clusters):
    model = KMeans(n_clusters=clusters, random_state=5)
    model

    # Fitting model
    model.fit(df)

    # Add a new class column to df_iris
    df["class"] = model.labels_

In [ ]:
test_cluster_amount(law_inc_df, 2)
law_inc_df.hvplot.scatter(x="deaths_rate_per_100k", y="lawtotal", by="class")

:NdOverlay   [class]
   :Scatter   [deaths_rate_per_100k]   (lawtotal)

In [ ]:
fig = px.scatter_3d(
    law_inc_df,
    x="deaths_rate_per_100k",
    y="lawtotal",
    z="elementary",
    color="class",
    symbol="class",
    width=800,
)
fig.update_layout(legend=dict(x=0, y=1))
fig.show()

In [ ]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,OneHotEncoder
import pandas as pd
import tensorflow as tf

# map drive to retrieve datasets
from google.colab import drive
drive.mount('/content/Drive')

# map drive to retrieve datasets
from google.colab import drive
drive.mount('/content/Drive')

# import dataset
law_inc_df = pd.read_csv('/content/Drive/MyDrive/Final_Project/Resources/gun_law_provisions_2020.csv')
law_inc_df

Drive already mounted at /content/Drive; to attempt to forcibly remount, call drive.mount("/content/Drive", force_remount=True).
Drive already mounted at /content/Drive; to attempt to forcibly remount, call drive.mount("/content/Drive", force_remount=True).


,state,year,felony,invcommitment,invoutpatient,danger,drugmisdemeanor,alctreatment,alcoholism,relinquishment,...,dvrosurrender,dvrosurrendernoconditions,dvrosurrenderdating,expartesurrender,expartesurrendernoconditions,expartesurrenderdating,dvroremoval,stalking,lawtotal,deaths_rate_per_100k
0,Alabama,2020,0,1,0,1,0,0,1,0,...,0,0,0,0,0,0,0,0,10,23.6
1,Alaska,2020,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,3,23.5
2,Arizona,2020,1,1,1,1,0,0,0,0,...,0,0,0,0,0,0,0,1,8,16.7
3,Arkansas,2020,1,1,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,11,22.6
4,California,2020,1,1,0,1,0,0,1,1,...,1,1,1,1,1,1,1,1,111,8.5
5,Colorado,2020,1,0,0,0,0,0,0,0,...,1,1,0,0,0,0,0,1,32,15.4
6,Connecticut,2020,1,1,1,1,1,0,0,1,...,1,1,1,1,1,1,0,1,89,6.0
7,Delaware,2020,1,1,1,0,1,0,0,0,...,0,0,0,0,0,0,0,1,41,14.4
8,Florida,2020,1,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,30,13.7
9,Georgia,2020,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,6,17.7


In [ ]:
# drop year
law_inc_df.drop(['year'], axis=1, inplace=True)

In [ ]:
# Generate our categorical variable list
law_cat = law_inc_df.dtypes[law_inc_df.dtypes == "object"].index.tolist()

In [ ]:
# Check the number of unique values in each column
law_inc_df[law_cat].nunique()

state    50
dtype: int64

Logistic Regression

In [ ]:
import numpy as np
import pandas as pd
from pathlib import Path
from collections import Counter

In [ ]:
# map drive to retrieve datasets
from google.colab import drive
drive.mount('/content/Drive')

# map drive to retrieve datasets
from google.colab import drive
drive.mount('/content/Drive')

# import dataset
law_inc_df = pd.read_csv('/content/Drive/MyDrive/Final_Project/Resources/gun_law_provisions_2020.csv')
law_inc_df.head()

Mounted at /content/Drive
Drive already mounted at /content/Drive; to attempt to forcibly remount, call drive.mount("/content/Drive", force_remount=True).


,state,year,felony,invcommitment,invoutpatient,danger,drugmisdemeanor,alctreatment,alcoholism,relinquishment,...,dvrosurrender,dvrosurrendernoconditions,dvrosurrenderdating,expartesurrender,expartesurrendernoconditions,expartesurrenderdating,dvroremoval,stalking,lawtotal,deaths_rate_per_100k
0,Alabama,2020,0,1,0,1,0,0,1,0,...,0,0,0,0,0,0,0,0,10,23.6
1,Alaska,2020,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,3,23.5
2,Arizona,2020,1,1,1,1,0,0,0,0,...,0,0,0,0,0,0,0,1,8,16.7
3,Arkansas,2020,1,1,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,11,22.6
4,California,2020,1,1,0,1,0,0,1,1,...,1,1,1,1,1,1,1,1,111,8.5


In [ ]:
# drop year column
law_inc_df.drop(['year'], axis=1, inplace=True)

In [ ]:
target = ["elementary"]
# Create our features
X = law_inc_df.drop('elementary', axis=1)
X = pd.get_dummies(X)

# Create our target
y = law_inc_df.loc[:, target].copy()

In [ ]:
X.describe()

,felony,invcommitment,invoutpatient,danger,drugmisdemeanor,alctreatment,alcoholism,relinquishment,violent,violenth,...,state_South Dakota,state_Tennessee,state_Texas,state_Utah,state_Vermont,state_Virginia,state_Washington,state_West Virginia,state_Wisconsin,state_Wyoming
count,50.000000,50.000000,50.000000,50.000000,50.000000,50.000000,50.000000,50.00000,50.000000,50.000000,...,50.000000,50.000000,50.000000,50.000000,50.000000,50.000000,50.000000,50.000000,50.000000,50.000000
mean,0.740000,0.600000,0.320000,0.540000,0.200000,0.120000,0.120000,0.14000,0.080000,0.080000,...,0.020000,0.020000,0.020000,0.020000,0.020000,0.020000,0.020000,0.020000,0.020000,0.020000
std,0.443087,0.494872,0.471212,0.503457,0.404061,0.328261,0.328261,0.35051,0.274048,0.274048,...,0.141421,0.141421,0.141421,0.141421,0.141421,0.141421,0.141421,0.141421,0.141421,0.141421
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.250000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,1.000000,1.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,1.000000,1.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.00000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [ ]:
# Check the balance of our target values
y.value_counts()

elementary
1             34
0             16
dtype: int64

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, 
                                                    y, 
                                                    random_state=1, 
                                                    stratify=y)
X_train.shape

(37, 185)

In [ ]:
# Resample the training data with the RandomOversampler
from imblearn.over_sampling import RandomOverSampler
ros = RandomOverSampler(random_state=1)
X_resampled, y_resampled = ros.fit_resample(X_train, y_train)

Counter(y_resampled)

Counter({'elementary': 1})

In [ ]:
# Train the Logistic Regression model using the resampled data
from sklearn.linear_model import LogisticRegression

model = LogisticRegression(solver='lbfgs', random_state=1)
model.fit(X_resampled, y_resampled)

/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


LogisticRegression(random_state=1)

In [ ]:
# Display the confusion matrix
from sklearn.metrics import confusion_matrix

y_pred = model.predict(X_test)
confusion_matrix(y_test, y_pred)

cm = confusion_matrix(y_test, y_pred)
cm_df = pd.DataFrame(
   cm, index=["Actual elementary law", "Actual not elementary law"],
   columns=["Predicted elementary law", "Predicted not elementary law"]
)
display(cm_df)

,Predicted elementary law,Predicted not elementary law
Actual elementary law,2,2
Actual not elementary law,1,8


In [ ]:
# Calculated the balanced accuracy score
from sklearn.metrics import balanced_accuracy_score

balanced_accuracy_score(y_test, y_pred)

0.6944444444444444

In [ ]:
# Print the imbalanced classification report
from imblearn.metrics import classification_report_imbalanced
print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

          0       0.67      0.50      0.89      0.57      0.67      0.43         4
          1       0.80      0.89      0.50      0.84      0.67      0.46         9

avg / total       0.76      0.77      0.62      0.76      0.67      0.45        13



In [ ]:
# Resample the training data with SMOTE
from imblearn.over_sampling import SMOTE
X_resampled, y_resampled = SMOTE(random_state=1, sampling_strategy='auto').fit_resample(
    X_train, y_train
)
Counter(y_resampled)

Counter({'elementary': 1})

In [ ]:
# Train the Logistic Regression model using the resampled data
model = LogisticRegression(solver='lbfgs', random_state=1)
model.fit(X_resampled, y_resampled)

/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


LogisticRegression(random_state=1)

In [ ]:
# Calculated the balanced accuracy score
y_pred = model.predict(X_test)
balanced_accuracy_score(y_test, y_pred)

0.6944444444444444

In [ ]:
# Display the confusion matrix
confusion_matrix(y_test, y_pred)

cm = confusion_matrix(y_test, y_pred)
cm_df = pd.DataFrame(
   cm, index=["Actual elementary law", "Actual not elementary law"],
   columns=["Predicted elementary law", "Predicted not elementary law"]
)
display(cm_df)

,Predicted elementary law,Predicted not elementary law
Actual elementary law,2,2
Actual not elementary law,1,8


In [ ]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

          0       0.67      0.50      0.89      0.57      0.67      0.43         4
          1       0.80      0.89      0.50      0.84      0.67      0.46         9

avg / total       0.76      0.77      0.62      0.76      0.67      0.45        13



In [ ]:
# Resample the data using the ClusterCentroids resampler
# Warning: This is a large dataset, and this step may take some time to complete
from imblearn.under_sampling import ClusterCentroids
cc = ClusterCentroids(random_state=1)
X_resampled, y_resampled = cc.fit_resample(X_train, y_train)
Counter(y_resampled)

Counter({'elementary': 1})

In [ ]:
# Train the Logistic Regression model using the resampled data
from sklearn.linear_model import LogisticRegression
model = LogisticRegression(solver='lbfgs', random_state=1)
model.fit(X_resampled, y_resampled)

/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


LogisticRegression(random_state=1)

In [ ]:
# Display the confusion matrix
from sklearn.metrics import confusion_matrix
y_pred = model.predict(X_test)
confusion_matrix(y_test, y_pred)

cm = confusion_matrix(y_test, y_pred)
cm_df = pd.DataFrame(
   cm, index=["Actual elementary law", "Actual not elementary law"],
   columns=["Predicted elementary law", "Predicted not elementary law"]
)
display(cm_df)

,Predicted elementary law,Predicted not elementary law
Actual elementary law,3,1
Actual not elementary law,3,6


In [ ]:
# Calculated the balanced accuracy score
from sklearn.metrics import balanced_accuracy_score
balanced_accuracy_score(y_test, y_pred)

0.7083333333333333

In [ ]:
# Print the imbalanced classification report
from imblearn.metrics import classification_report_imbalanced
print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

          0       0.50      0.75      0.67      0.60      0.71      0.50         4
          1       0.86      0.67      0.75      0.75      0.71      0.50         9

avg / total       0.75      0.69      0.72      0.70      0.71      0.50        13



In [ ]:
# Resample the training data with SMOTEENN

from imblearn.combine import SMOTEENN

smote_enn = SMOTEENN(random_state=0)
X_resampled, y_resampled = smote_enn.fit_resample(X, y)
Counter(y_resampled)

Counter({'elementary': 1})

In [ ]:
# Train the Logistic Regression model using the resampled data
from sklearn.linear_model import LogisticRegression
model = LogisticRegression(solver='lbfgs', random_state=1)
model.fit(X_resampled, y_resampled)

/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


LogisticRegression(random_state=1)

In [ ]:
# Display the confusion matrix
from sklearn.metrics import confusion_matrix

y_pred = model.predict(X_test)
confusion_matrix(y_test, y_pred)

cm = confusion_matrix(y_test, y_pred)
cm_df = pd.DataFrame(
   cm, index=["Actual elementary law", "Actual not elementary law"],
   columns=["Predicted elementary law", "Predicted not elementary law"]
)
display(cm_df)

,Predicted elementary law,Predicted not elementary law
Actual elementary law,4,0
Actual not elementary law,4,5


In [ ]:
# Calculated the balanced accuracy score
from sklearn.metrics import balanced_accuracy_score

balanced_accuracy_score(y_test, y_pred)

0.7777777777777778

In [ ]:
# Print the imbalanced classification report
from imblearn.metrics import classification_report_imbalanced

print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

          0       0.50      1.00      0.56      0.67      0.75      0.58         4
          1       1.00      0.56      1.00      0.71      0.75      0.53         9

avg / total       0.85      0.69      0.86      0.70      0.75      0.55        13

